In [1]:
%gui qt

In [2]:
def start_imagej():
    import imagej
    _ij = imagej.init(headless=False)

    from jnius import PythonJavaClass, java_method, autoclass
    class JavaRunnable(PythonJavaClass):
        __javainterfaces__ = ['java/lang/Runnable']

        def __init__(self, f):
            super(JavaRunnable, self).__init__()
            self._f = f

        @java_method('()V')
        def run(self):
            self._f()


    class HackedUIService(object):
        def __init__(self, ij):
            super(HackedUIService, self).__init__()
            self._ij = ij
        def __getattr__(self, name):
            value = getattr(self._ij.ui(), name)
            if str(type(value)) in ['jnius.JavaMethod', 'jnius.JavaMultipleMethod']:
                return lambda: self._ij.thread().queue(JavaRunnable(lambda: value(kwargs)))
            return value
    p_ui = HackedUIService(_ij)
    _ij.ui = lambda: p_ui

    _ij.ui().showUI()
    global ij
    ij = _ij

from PyQt5 import QtCore
QtCore.QTimer.singleShot(0, start_imagej)
